In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import norm
from scipy.fftpack import fft
from scipy.io import wavfile

% matplotlib inline

# Récupération du signal audio

In [6]:
X = wavfile.read('data/aeiou.wav')
T = len(X)

# Estimateur

In [14]:
def generation_est(X, d, T, mu):

    theta_est = np.matrix(np.zeros((d,T)))
    X_est = np.zeros(T)
    
    for k in (np.arange(T)): # on fait T itérations
        XX = X[k-d:k][::-1]
        
        if k==0:
            theta_est[:,k] = np.matrix(mu * X[k] * XX / (1 + mu * norm(XX) ** 2)).T 
        else:
            theta_est[:,k] = np.matrix(theta_est[:,k-1].T + mu * (X[k] - np.dot(XX, theta_est[:,k-1])) * XX / (1 + mu * norm(XX) ** 2)).T  

        X_est[k] = np.dot(XX, theta_est[:,k])
        
    return X_est, theta_est

In [8]:
#FACTORISATION EN FONCTION
def aggregation(X, estimations, T, eta, strategy = 1):
    
    """Retourne un predicteur par aggregation"""
    
    N = len(estimations[:,0])
    pred = np.zeros(T)
    alpha = 1./N * np.ones(N)
    
    for t in np.arange(T):
    
        if t > 0:
            v = np.zeros(N)
            for i in np.arange(N):
                #On choisit la stratégie la plus efficace selon les hypothèses sur le bruit
                #Voir le rapport pour plus de détails à ce propos
                if strategy == 1:
                    v[i] = alpha[i] * np.exp(-2*eta*(pred[t-1]-X[t-1])*estimations[i,t-1])
                else:
                    v[i] = alpha[i] * np.exp(-eta*(estimations[i,t-1]-X[t-1])**2)
            alpha = v / np.sum(v)
            
        pred[t] = np.dot(alpha, estimations[:,t])
            
    return pred, alpha

In [15]:
# génération des estimateurs
log_mu = np.arange(-3, 0, 0.3)
N = len(log_mu)
mu = np.power(10, log_mu)
d = 20 # ordre du TVAR
estimations = np.matrix(np.zeros((N, T)))

for k in np.arange(N):
    estimations[k,:], theta_est = generation_est(X, d, T, mu[k])

C:\Users\Amaury\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


error: (offx>=0 && offx<len(x)) failed for 2nd keyword offx: dnrm2:offx=0

# Analyse spectrale

In [ ]:
def dsp(theta, T, N = 512):
    dsp_array = np.matrix(np.zeros((N, T)))
    
    for t in np.arange(T):
        dsp_array[:,t] = np.matrix(1./abs(fft(theta[:,t].T, N)) ** 2).T
        
    lambd = np.arange(N, dtype = 'double')/N
        
    return lambd, dsp_array